In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import welch, csd, freqz

from speckit import compute_spectrum
from speckit.noise import butter_lowpass, butter_lowpass_filter

# --- 1. Define Filter and Generate Signals ---
order = 3
fs = 2
cutoff = 1e-3

# Plot the filter's frequency response for validation
b, a = butter_lowpass(cutoff, fs, order)
w, h = freqz(b, a, worN=8000)

fig, ax = plt.subplots(figsize=(8, 3), dpi=120)
ax.loglog(0.5 * fs * w / np.pi, np.abs(h), "b")
ax.axvline(cutoff, color="k", linestyle="--", label=f"Cutoff = {cutoff} Hz")
ax.set_title("Lowpass Filter Frequency Response")
ax.set_xlabel("Frequency [Hz]")
ax.set_ylabel("Gain")
ax.grid(True, which="both", linestyle=":")
ax.legend()
plt.show()

# Generate a white noise input and a filtered output
N = int(1e6)
np.random.seed(42)  # For reproducibility
x1 = np.random.randn(N)
x2 = butter_lowpass_filter(x1, cutoff, fs, order)

# Plot a snippet of the time-series
fig, ax = plt.subplots(figsize=(8, 4), dpi=120)
ax.set_xlabel("Sample")
ax.set_ylabel("Signal")
ax.plot(x1[:5000], color="gray", label="Input Signal (x1, white noise)", alpha=0.8)
ax.plot(x2[:5000], color="k", label="Filtered Signal (x2, red noise)")
ax.legend()
plt.show()


# --- 2. Compute Reference Spectra with Welch's Method ---
nperseg_welch = int(N / 8)
f_welch_ps1, ps1_welch = welch(x1, fs=fs, nperseg=nperseg_welch, window="hann")
f_welch_ps2, ps2_welch = welch(x2, fs=fs, nperseg=nperseg_welch, window="hann")
f_welch_csd, csd12_welch = csd(x1, x2, fs=fs, nperseg=nperseg_welch, window="hann")

# --- 3. Compute Spectra with the New speckit API ---
print("Computing cross-spectrum with speckit...")
result = compute_spectrum(
    data=[x1, x2],
    fs=fs,
    Lmin=int(10e3),
    order=-1,
    win="hann",
    olap=0.5,
)
print("Computation complete.")


# --- 4. Plot and Compare Results ---
fig, ax = plt.subplots(figsize=(6, 4), dpi=150)
ax.set_xlabel("Fourier Frequency (Hz)")
ax.set_ylabel(r"Spectral Densities (${\rm units}^2/{\rm Hz}$)")

# Plot Welch reference lines
ax.loglog(f_welch_ps1, ps1_welch, label="PSD(x1) (Welch)", color="C0", alpha=0.5)
ax.loglog(f_welch_ps2, ps2_welch, label="PSD(x2) (Welch)", color="C1", alpha=0.5)
ax.loglog(
    f_welch_csd,
    np.abs(csd12_welch),
    label="|CSD(x1,x2)| (Welch)",
    color="C2",
    alpha=0.5,
)

# Plot speckit results using the properties of the SpectrumResult object
ax.loglog(result.f, result.Gxx, label="PSD(x1) (speckit)", color="C0", ls="--")
ax.loglog(result.f, result.Gyy, label="PSD(x2) (speckit)", color="C1", ls="--")
ax.loglog(
    result.f, np.abs(result.Gxy), label="|CSD(x1,x2)| (speckit)", color="C2", ls="--"
)

ax.legend()
ax.set_title("speckit vs. Welch's Method")
fig.tight_layout()
plt.show()